In [1]:
#input tensorflow and keras

import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D, Convolution2D
from keras.optimizers import SGD, Adam, RMSprop
from keras.metrics import categorical_accuracy
from keras import regularizers
from PIL import Image

import numpy as np
import pandas as pd
import random
import tensorflow as tf
from PIL import Image

import matplotlib
import matplotlib.pyplot as plt

import os
print(os.listdir("../input/inceptionresnetv2"))

# #Online code use for colab
# from google.colab import drive
# drive.mount('/content/drive')
# config = tf.ConfigProto()
# config.gpu_options.allow_growth = True


Using TensorFlow backend.


['imagenet_class_index.json', 'inception_resnet_v2_weights_tf_dim_ordering_tf_kernels.h5', 'inception_resnet_v2_weights_tf_dim_ordering_tf_kernels_notop.h5']


In [2]:
#Set seed
seed = 1990
random.seed(seed)
np.random.seed(seed)
tf.set_random_seed(seed)

In [3]:
#append text .png at the end
def append_ext(fn):
    return fn+".png"

#convert int to str
def convert_int(i):
    return str(i)

# data preprocessing
image_path = "../input/aptos2019-blindness-detection/train_images"
traindf=pd.read_csv("../input/aptos2019-blindness-detection/train.csv")
traindf["id_code"]=traindf["id_code"].apply(append_ext)
traindf["diagnosis"]=traindf["diagnosis"].apply(convert_int)

#build the generator
train_datagen = ImageDataGenerator(
#     rotation_range=10,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
    rescale = 1./255, 
    validation_split=0.8,
    horizontal_flip=True,
#     fill_mode='nearest',
#     zca_whitening=True,
#     shear_range=0.2,
#     zoom_range=0.2,
)

In [4]:
#Set batch size
batch_size=32

# flow_from datafreame format
train_data = train_datagen.flow_from_dataframe(
    dataframe=traindf,
    directory=image_path,
    x_col="id_code",
    y_col="diagnosis",
    target_size=(300, 300),
    color_mode='rgb',
    subset='training',
    batch_size=batch_size
)

validation_data = train_datagen.flow_from_dataframe(
    dataframe=traindf,
    directory=image_path,
    x_col="id_code",
    y_col="diagnosis",
    target_size=(300, 300),
    color_mode='rgb',
    subset='validation',
    batch_size=batch_size
)

# flow from directory format
# train_data = train_datagen.flow_from_directory(
#     directory=image_path,
#     target_size=(300, 300),
#     color_mode='rgb',
#     subset='training',
#     batch_size=batch_size
# )

# validation_data = train_datagen.flow_from_directory(
#     directory=image_path,
#     target_size=(300, 300),
#     color_mode='rgb',
#     subset='validation',
#     batch_size=batch_size
# )

Found 733 validated image filenames belonging to 5 classes.
Found 2929 validated image filenames belonging to 5 classes.


In [5]:
#input keras applications
from keras.applications import VGG16
from keras.applications import ResNet50
from keras.applications import NASNetLarge
from keras.applications import InceptionResNetV2

#VGG-16 model
VGG16_weights_path = '../input/vgg16/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'
vgg16 = VGG16(include_top=False,input_shape=(300, 300, 3),weights = VGG16_weights_path,classes=5)

model = Sequential()
model.add(vgg16)
model.add(Flatten())
model.add(Dense(256, activation='relu',activity_regularizer=regularizers.l2(0.001)))
model.add(Dense(256, activation='relu',activity_regularizer=regularizers.l2(0.001)))
model.add(Dropout(0.25))
model.add(Dense(5, activation='softmax',activity_regularizer=regularizers.l1(0.001)))
model.summary()

# resNet50 model
# resnet_weights_path = '../input/resnet50/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'
# resNet50 = ResNet50(include_top = False, pooling = 'avg', weights = resnet_weights_path,classes=5)

# model = Sequential()
# model.add(resNet50)
# model.add(Dense(5, activation='softmax',activity_regularizer=regularizers.l1(0.001)))
# model.summary()

# NasNetLarge model
# NASNetLarge_weights_path = '../input/nasnetlarge/NASNet-large-no-top.h5'
# NasNetLarge = NASNetLarge(include_top = False, pooling = 'avg', weights = NASNetLarge_weights_path,input_shape=(300, 300, 3),classes=5)

# model = Sequential()
# model.add(NasNetLarge)
# model.add(Dense(5, activation='softmax',activity_regularizer=regularizers.l1(0.001)))
# model.summary()

# InceptionResNetV2 model
# inceptionResNetV2_path = '../input/inceptionresnetv2/inception_resnet_v2_weights_tf_dim_ordering_tf_kernels_notop.h5'
# inceptionResNetV2 = InceptionResNetV2(include_top=False,input_shape=(300, 300, 3),weights = inceptionResNetV2_path,classes=5)

# model = Sequential()
# model.add(inceptionResNetV2)
# model.add(Flatten())
# model.add(Dense(256, activation='relu',activity_regularizer=regularizers.l2(0.001)))
# model.add(Dense(256, activation='relu',activity_regularizer=regularizers.l2(0.001)))
# model.add(Dropout(0.25))
# model.add(Dense(5, activation='softmax',activity_regularizer=regularizers.l1(0.001)))
# model.summary()

#Set optimizer
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
rmsprop = RMSprop(lr=1e-4, rho=0.9, epsilon=None, decay=0.0)
adam = Adam(lr=0.0001)

#model compile
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['categorical_accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 9, 9, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 41472)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               10617088  
_________________________________________________________________
dense_2 (Dense)              (None, 256)               65792     
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 5)                 1285      
Total params: 25,398,853
Trainable params: 25,398,853
Non-trainable params: 0
________________________________________________________________

In [6]:
#Call back
from keras.callbacks import TensorBoard
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping

tbCallBack = TensorBoard(log_dir='log',histogram_freq=0,write_graph=True,write_images=False)
checkpoint = ModelCheckpoint(filepath='best_aptos_model.hdf5',save_best_only='True',monitor='val_loss')
earlyStop = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=0, mode='auto')


In [7]:
#Start training....... fit the model
model.fit_generator(
        train_data,
        steps_per_epoch=len(train_data),
        verbose=1, 
        epochs=200,
        validation_data=validation_data,
        validation_steps=len(validation_data),
        callbacks=[tbCallBack,checkpoint,earlyStop] #<-- add earlyStop to get the best model from call back
) 

Epoch 1/200
23/23 [==============================] - 395s 17s/step - loss: 1.6505 - categorical_accuracy: 0.6033 - val_loss: 1.3978 - val_categorical_accuracy: 0.6784
Epoch 2/200
23/23 [==============================] - 345s 15s/step - loss: 1.3399 - categorical_accuracy: 0.6998 - val_loss: 1.3328 - val_categorical_accuracy: 0.6709
Epoch 3/200
23/23 [==============================] - 342s 15s/step - loss: 1.2887 - categorical_accuracy: 0.6997 - val_loss: 1.2850 - val_categorical_accuracy: 0.6951
Epoch 4/200
23/23 [==============================] - 349s 15s/step - loss: 1.2428 - categorical_accuracy: 0.7006 - val_loss: 1.2706 - val_categorical_accuracy: 0.7132
Epoch 5/200
23/23 [==============================] - 346s 15s/step - loss: 1.2206 - categorical_accuracy: 0.7076 - val_loss: 1.2364 - val_categorical_accuracy: 0.6978
Epoch 6/200
23/23 [==============================] - 343s 15s/step - loss: 1.1950 - categorical_accuracy: 0.7161 - val_loss: 1.2019 - val_categorical_accuracy: 0.703

Epoch 50/200
23/23 [==============================] - 371s 16s/step - loss: 0.3655 - categorical_accuracy: 0.9837 - val_loss: 0.8918 - val_categorical_accuracy: 0.7562
Epoch 51/200
23/23 [==============================] - 369s 16s/step - loss: 0.3617 - categorical_accuracy: 0.9878 - val_loss: 0.8896 - val_categorical_accuracy: 0.7436
Epoch 52/200
23/23 [==============================] - 373s 16s/step - loss: 0.3542 - categorical_accuracy: 0.9849 - val_loss: 0.8598 - val_categorical_accuracy: 0.7614
Epoch 53/200
23/23 [==============================] - 370s 16s/step - loss: 0.3459 - categorical_accuracy: 0.9851 - val_loss: 0.8680 - val_categorical_accuracy: 0.7576
Epoch 54/200
23/23 [==============================] - 365s 16s/step - loss: 0.3333 - categorical_accuracy: 0.9878 - val_loss: 0.9052 - val_categorical_accuracy: 0.7446
Epoch 55/200
23/23 [==============================] - 358s 16s/step - loss: 0.3292 - categorical_accuracy: 0.9851 - val_loss: 0.8854 - val_categorical_accuracy:

In [ ]:
#Evaluate
model.load_weights('best_aptos_model.hdf5')
loss_and_metrics =model.evaluate_generator(validation_data,len(validation_data))
print('Test loss:{}\nTest accuracy:{}'.format(loss_and_metrics[0], loss_and_metrics[1]))

In [ ]:
#testing the model
from keras_preprocessing.image import ImageDataGenerator

testdf=pd.read_csv("../input/aptos2019-blindness-detection/sample_submission.csv")
testdf["id_code"]=testdf["id_code"].apply(append_ext)
testdf["diagnosis"]=testdf["diagnosis"].apply(convert_int)
image_path = "../input/aptos2019-blindness-detection/test_images"

test_datagen=ImageDataGenerator(rescale=1./255.)
test_generator = test_datagen.flow_from_dataframe(
    dataframe=testdf,
    directory=image_path,
    x_col="id_code",
    y_col="diagnosis",
    target_size=(300, 300),
    color_mode='rgb',
    shuffle=False,
    batch_size=batch_size
)

test_generator.reset()
test_pred=model.predict_generator(test_generator,steps=len(test_generator),verbose=1)



In [ ]:

print(test_pred)
predicted_class=np.argmax(test_pred,axis=1)

print(predicted_class)
import collections
print(collections.Counter(predicted_class))

sample = pd.read_csv("../input/aptos2019-blindness-detection/sample_submission.csv")
sample.diagnosis = predicted_class.astype(int)
sample.to_csv("submission.csv", index=False)